In [52]:
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import pandas as pd
# import rpy2_arrow.pyarrow_rarrow as pyra
%load_ext rpy2.ipython
rpy2.robjects.numpy2ri.activate()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [140]:
%%R
library(lsa)
library(vegan)
library(ape)
# library(psych)
library(ggplot2)
# library(dendextend)
# library(circlize)
# library(grid)
library(tidyr)

## feature list humans (leuven) vs feature list gpt

### old code

In [30]:
%%R
# set working dir to the dir the script is in
setwd("/home/siddsuresh97/Projects/conceptual_representations_gpt/emnlp/")

# Load data
leuven_animals = read.csv('data/feature_listing/animal_leuven_norms.csv')
leuven_tools = read.csv('data/feature_listing/artifacts_leuven_norms.csv')
clip = read.csv('data/clip/clip_embeddings.csv')
w2v = read.csv('data/word2vec/word2vec_embeddings.csv')
human_triplet_mds = read.csv('data/triplets/summer_data/human_reptile_tool_embedding_df.csv')
gpt_triplet_mds = read.csv('data/triplets/summer_data/gpt_reptile_tool_embedding_df_temp0_cogsci2023.csv')
gpt_embed <- read.csv('data/gpt/gpt_embeddings_text-similarity-davinci-001.csv')

In [41]:
%%R 
# clip_data
clip = clip[order(clip$X), ]
clip_dsm =1-lsa::cosine(t(data.matrix(clip[,2:ncol(clip)])))
clip_mds =  cmdscale(clip_dsm,k = 3)
rownames(clip_dsm) <- clip$X
rownames(clip_mds) <- clip$X
clip_mds <- clip_mds[order(rownames(clip_mds)),]

In [42]:
%%R 
# gpt_embed_data
gpt_embed = gpt_embed[order(gpt_embed$X), ]
gpt_embed_dsm =1-lsa::cosine(t(data.matrix(gpt_embed[,2:ncol(gpt_embed)])))
gpt_embed_mds =  cmdscale(gpt_embed_dsm,k = 3)
rownames(gpt_embed_dsm) <- gpt_embed$X
rownames(gpt_embed_mds) <- gpt_embed$X
gpt_embed_mds <- gpt_embed_mds[order(rownames(gpt_embed_mds)),]

In [43]:
%%R 
#word2vec_data
w2v = w2v[order(w2v$X), ]
w2v_dsm =1-lsa::cosine(t(data.matrix(w2v[,2:ncol(w2v)])))
# <!-- w2v_dsm = dist(w2v[,2:ncol(w2v)) -->
w2v_mds =  cmdscale(w2v_dsm,k = 3)
rownames(w2v_dsm) <- w2v$X
rownames(w2v_mds) <- w2v$X
w2v_mds <- w2v_mds[order(rownames(w2v_mds)),]

In [44]:
%%R 
#leuven_data
leuven = merge(leuven_animals, leuven_tools, by = 'X', all = TRUE)
leuven[is.na(leuven)] = 0
leuven_cosine_dist = 1-  lsa::cosine(t(data.matrix(leuven[,2:ncol(leuven)])))
leuven_mds <- cmdscale(leuven_cosine_dist,k = 3)
rownames(leuven_cosine_dist) <- leuven$X
rownames(leuven_mds) <- leuven$X
leuven_mds <- leuven_mds[order(rownames(leuven_mds)),]

In [45]:
%%R 
#flan_xl_verified
flan_xl_verified = read.csv('data/feature_listing/flan_xl_wide.csv')
flan_xl_dsm = 1-  lsa::cosine(t(data.matrix(flan_xl_verified[,2:ncol(flan_xl_verified)])))
flan_xl_mds <- cmdscale(flan_xl_dsm,k = 3)
rownames(flan_xl_dsm) <- flan_xl_verified$Concept
rownames(flan_xl_mds) = rownames(flan_xl_dsm)
flan_xl_mds <- flan_xl_mds[order(rownames(flan_xl_mds)),]

In [46]:
%%R 
#flan_xxl_verified
flan_xxl_verified = read.csv('data/feature_listing/flan_xxl_wide.csv')
flan_xxl_dsm = 1-  lsa::cosine(t(data.matrix(flan_xxl_verified[,2:ncol(flan_xxl_verified)])))
flan_xxl_mds <- cmdscale(flan_xxl_dsm,k = 3)
rownames(flan_xxl_dsm) <- flan_xxl_verified$Concept
rownames(flan_xxl_mds) = rownames(flan_xxl_dsm)
flan_xxl_mds <- flan_xxl_mds[order(rownames(flan_xxl_mds)),]

In [47]:
%%R 
#gpt unverified
gpt_summer_no_mc = read.csv('data/feature_listing/gpt_response_unverified_wide_no_matrix_completion.csv')
unverified_gpt_dsm = 1-  lsa::cosine(t(data.matrix(gpt_summer_no_mc[,2:ncol(gpt_summer_no_mc)])))
unverified_gpt_mds_summer_no_mc <- cmdscale(unverified_gpt_dsm,k = 3)
rownames(unverified_gpt_dsm) <- gpt_summer_no_mc$Concept
rownames(unverified_gpt_mds_summer_no_mc) = rownames(unverified_gpt_dsm)
unverified_gpt_mds_summer_no_mc <- unverified_gpt_mds_summer_no_mc[order(rownames(unverified_gpt_mds_summer_no_mc)),]

In [48]:
%%R 
# gpt verified 
gpt_summer_no_mc = read.csv('data/feature_listing/gpt_response_yes_wide_no_matrix_completion.csv')
verified_gpt_dsm = 1-  lsa::cosine(t(data.matrix(gpt_summer_no_mc[,2:ncol(gpt_summer_no_mc)])))
verified_gpt_mds_summer_no_mc <- cmdscale(verified_gpt_dsm,k = 3)
rownames(verified_gpt_dsm) <- gpt_summer_no_mc$Concept
rownames(verified_gpt_mds_summer_no_mc) = rownames(verified_gpt_dsm)
verified_gpt_mds_summer_no_mc <- verified_gpt_mds_summer_no_mc[order(rownames(verified_gpt_mds_summer_no_mc)),]

In [49]:
%%R 
# human triplets
rownames(human_triplet_mds) <- human_triplet_mds$concept
human_triplet_mds <- human_triplet_mds[order(rownames(human_triplet_mds)),]
human_triplet_mds <- human_triplet_mds[, 3:5]
human_triplet_cosine_dist = 1-  lsa::cosine(t(data.matrix(human_triplet_mds)))
rownames(human_triplet_cosine_dist) <- human_triplet_mds$concept

In [50]:
%%R 
# gpt triplets
rownames(gpt_triplet_mds) <- gpt_triplet_mds$concept
gpt_triplet_mds <- gpt_triplet_mds[order(rownames(gpt_triplet_mds)),]
gpt_triplet_mds <- gpt_triplet_mds[, 3:5]
gpt_triplet_cosine_dist = 1-  lsa::cosine(t(data.matrix(gpt_triplet_mds)))
rownames(gpt_triplet_cosine_dist) <- gpt_triplet_mds$concept

### new code

In [125]:
%%R
gpt_003_pairwise_dsm = read.csv('data/pairwise/text-davinci-003/response_davinci-003_0.7_mean_matrix.csv')
gpt_003_pairwise_dsm = gpt_003_pairwise_dsm[1:(nrow(gpt_003_pairwise_dsm)), 2:(ncol(gpt_003_pairwise_dsm)), drop = FALSE]
colnames(gpt_003_pairwise_dsm) <- NULL
gpt_003_pairwise_dsm = (t(as.matrix(gpt_003_pairwise_dsm)) + as.matrix(gpt_003_pairwise_dsm))/2
gpt_003_pairwise_dsm = 7 - gpt_003_pairwise_dsm
gpt_003_pairwise_mds = cmdscale(as.matrix(gpt_003_pairwise_dsm),k = 3)

In [149]:
human_pairwise_dsm = pd.read_csv('data/pairwise/human/human_rating_df.csv')
# keep columns ['concept_1', 'concept_2', 'response_clean']
human_pairwise_dsm = human_pairwise_dsm[['concept_1', 'concept_2', 'response_clean']]

# group by concept_1 and concept_2 and take mean of response_clean
human_pairwise_dsm = human_pairwise_dsm.groupby(['concept_1', 'concept_2']).mean().reset_index()
human_pairwise_dsm = pd.pivot(human_pairwise_dsm, index = 'concept_1', columns = 'concept_2', values = 'response_clean')
# wherever there is a nan, replace with 7
human_pairwise_dsm = human_pairwise_dsm.fillna(6)

#remove row and column names
human_pairwise_dsm.index.name = None
human_pairwise_dsm.columns.name = None

# convert from python to R
def numpy_mat_to_R(var_name, var):
    nr,nc = var.shape
    Br = ro.r.matrix(var, nrow=nr, ncol=nc)
    ro.r.assign(var_name, Br)
numpy_mat_to_R('human_pairwise_dsm', human_pairwise_dsm.values)



In [151]:
%%R 
human_pairwise_dsm = (t(as.matrix(human_pairwise_dsm)) + as.matrix(human_pairwise_dsm))/2
human_pairwise_dsm = 6 - human_pairwise_dsm
human_pairwise_mds = cmdscale(human_pairwise_dsm,k = 3)

## pairwise protest all elements

In [152]:
%%R 

matrices = list(leuven_mds, w2v_mds, clip_mds, unverified_gpt_mds_summer_no_mc, verified_gpt_mds_summer_no_mc, human_triplet_mds, gpt_triplet_mds, gpt_embed_mds, flan_xl_mds, flan_xxl_mds, human_pairwise_mds, gpt_003_pairwise_mds)
## Create an empty list to store the results
results = list()

## Iterate through all combinations of the matrices
for (i in 1:(length(matrices))) {
    for (j in 1:(length(matrices))) {
        # Pass the two matrices to the protest function and store the result
        result = protest(matrices[[i]], matrices[[j]], symmetric=TRUE)
        results = c(results, sqrt(1-result$ss))
    }
}

## make the results into a table
## Create a data frame with all possible combinations of matrix names
matrix_names = list("leuven", "w2v", "clip", "unverified_gpt_summer_no_mc", "verified_gpt_summer_no_mc", "human_triplet", "gpt_triplet", "gpt_embed_mds", "flan_xl_mds", "flan_xxl_mds", "human_pairwise_mds", "gpt_003_pairwise_mds")
matrix_combinations = expand.grid(matrix_names, matrix_names)

## Assign column names to the data frame
colnames(matrix_combinations) = c("matrix1", "matrix2")

## Fill in the values
k = 1
for (i in 1:length(matrix_names)) {
    for (j in 1:length(matrix_names)) {
        matrix_combinations[k, "result"] = results[k]
        k = k + 1
    }
}

protest_results =  pivot_wider(matrix_combinations, names_from = matrix1, values_from = result)
df <- apply(protest_results,2,as.character)

write.table(data.frame(df), "results/correlation_table.csv", sep=",")